# Работа с API YandexGPT

In [1]:
import requests
import json
import time

Для работы с API YandexGPT необходим API-ключ для сервисного аккаунта Яндекс.Облака с ролями 'ai.languageModels.user' и 'ai.imageGeneration.user'.

Подробнее о создании API-ключа: https://yandex.cloud/ru/docs/iam/concepts/authorization/api-key

In [2]:
yandex_cloud_catalog = "b1glihj9h7pkj7mnd6at"
yandex_api_key = "AQVNyJRotHlHIhGec5YfWrslmo8tsbsc8eatOf_V"

Варианты модели:
- yandexgpt-lite
- yandexgpt
- summarization (для суммарищации текстов)

In [21]:
yandex_gpt_model = "yandexgpt"

Структура промта:
- Общая инструкция (Ты ...)
- Запрос (Напиши ...)
- Контекст (Текст)
- Формат ответа

Параметры генерации
- temperature: чем выше значение этого параметра, тем более креативными и случайными будут ответы модели. Принимает значения от 0 до 1

In [24]:
temperature = 0.7
prompt = "Ты seniour программист. Напиши самую креативную, но работающую программу на python полностью. В ответе напиши программу и какую задачу она выполняет."

In [25]:
body = {
    "modelUri": f"gpt://{yandex_cloud_catalog}/{yandex_gpt_model}",
    "completionOptions": {
        "stream": False,
        "temperature": temperature,
        "maxTokens": "5000",
    },
    "messages": [{"role": "user", "text": prompt}],
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {yandex_api_key}",
}

response = requests.post(url, headers=headers, json=body)
response_json = json.loads(response.text)
answer = response_json["result"]["alternatives"][0]["message"]["text"]
print(answer)

Вот программа, которая генерирует случайную мелодию, используя заметки из одной октавы. Вы можете ввести количество нот и их длительность, и программа постарается создать приятную мелодию.

Задача программы — синтезировать мелодию на основе введённых данных о нотах и их длительности. Это может быть полезно для тех, кто хочет создать свою собственную мелодию или просто развлечься.

Код программы:
```
import random

notes = ["до", "ре", "ми", "фа", "соль", "ля", "си"]
durations = [1, 2, 4, 8]

def generate_melody(number_of_notes, duration):
    for i in range(number_of_notes):
        note = random.choice(notes)
        duration_value = random.choice(durations)
        print(f"{note} ({duration_value})")
        sleep(duration)

if __name__ == '__main__':
    number_of_notes = int(input("Введите количество нот: "))
    duration = int(input("Укажите длительность в секундах: "))

    generate_melody(number_of_notes=number_of_notes, duration=duration)
```
Обратите внимание, что эта программ

### Добавления системного промпта

In [15]:
system_prompt = "Исправь ошибки в тексте. В ответе напиши только исправленный текст."
prompt = "Сабака гуляет по трове а люди набблюдают"

In [16]:
import requests
import json

body = {
    "modelUri": f"gpt://{yandex_cloud_catalog}/{yandex_gpt_model}",
    "completionOptions": {"stream": False, "temperature": 0, "maxTokens": "2000"},
    "messages": [
        {"role": "system", "text": system_prompt},
        {"role": "user", "text": prompt},
    ],
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {yandex_api_key}",
    "x-folder-id": yandex_cloud_catalog,
}

response = requests.post(url, headers=headers, json=body)
response_json = json.loads(response.text)
answer = response_json["result"]["alternatives"][0]["message"]["text"]
print(answer)

Собака гуляет по траве, а люди наблюдают.


### Работа в асинхронном режиме

Выполнение запроса на генерацию текста в асинхронном режиме. Генерация может выполняться несколько секунд. В результате выполнения запроса получаем operation_id, с помощью которого можно получать результат, выполнив отдельный запрос

In [17]:
body = {
    "modelUri": f"gpt://{yandex_cloud_catalog}/{yandex_gpt_model}",
    "completionOptions": {"stream": False, "temperature": 0, "maxTokens": "2000"},
    "messages": [
        {"role": "system", "text": system_prompt},
        {"role": "user", "text": prompt},
    ],
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completionAsync"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {yandex_api_key}",
    "x-folder-id": yandex_cloud_catalog,
}

response = requests.post(url, headers=headers, json=body)
response_json = json.loads(response.text)
print(response_json)
operation_id = response_json["id"]

{'id': 'd7q3sfv78c22vkr181d1', 'description': 'Async GPT Completion', 'createdAt': '2024-04-14T12:19:02Z', 'createdBy': 'aje4pspgk8umd6h29st2', 'modifiedAt': '2024-04-14T12:19:02Z', 'done': False, 'metadata': None}


Необходимо дождаться выполнения генерации

In [18]:
url = f"https://llm.api.cloud.yandex.net/operations/{operation_id}"
headers = {"Authorization": f"Api-Key {yandex_api_key}"}

while True:
    response = requests.get(url, headers=headers)
    response_json = json.loads(response.text)
    done = response_json["done"]
    if done:
        break
    else:
        time.sleep(0.5)

answer = response_json["response"]["alternatives"][0]["message"]["text"]
print(answer)

Собака гуляет по траве, а люди наблюдают.
